In [1]:
# get our db url
# create db engine via sqlalchemy

In [2]:
from env import config, BASE_DIR
from sqlalchemy import create_engine, text

In [7]:
DATABASET_URL = config("FLIGHT_PRICES_DATABSE_URL", default=None)
DATABASET_URL is not None

True

In [8]:
engine = create_engine(str(DATABASE_URL))

In [13]:
DATASET_DIR = BASE_DIR / "datasets"
DATASET_DIR.exists()

True

In [14]:
GRETEL_DIR = DATASET_DIR / "gretel"
GRETEL_DIR.exists()



True

In [15]:
import pandas as pd # dask for larger datasets
# from dask import dataframe as dd

In [16]:
table_name = "flight_prices"
df = pd.read_sql(table_name, engine, index_col="index")
df.tail(n=1)
# out_df.to_sql(table_name, engine, if_exists='replace', index=True)



,flightDate,startingAirport,destinationAirport,isBasicEconomy,isRefundable,isNonStop,segmentsAirlineName,totalFare
index,,,,,,,,
9591,2022-04-22,IAD,LAX,1,0,1,United,221


In [17]:
last_row = df.tail(n=1)
last_idx = last_row.index[0]
last_idx



np.int64(9591)

In [18]:
max_flight_date = df.flightDate.max() # .date()
max_flight_date



Timestamp('2022-04-22 00:00:00')

In [19]:
import datetime



In [20]:
gretel_dfs = []
for path in GRETEL_DIR.glob("*.gz"):
    with gzip.open(path) as f:
        gretel_dfs.append(pd.read_csv(f))
new_df = pd.concat(gretel_dfs)
new_df.index = new_df.index + last_idx + 1 
new_df['flightDate'] = pd.to_datetime(new_df['flightDate'])
new_df['flightDate'] = new_df['flightDate'].apply(lambda x: max_flight_date + (max_flight_date - x) + datetime.timedelta(days=1))
# del new_df['totalFare']
new_df.head()



ValueError: No objects to concatenate

In [13]:


new_df.flightDate.max().date()



,flightDate,startingAirport,destinationAirport,totalFare
0,2022-04-17,ATL,BOS,248.6
1,2022-04-17,ATL,BOS,248.6
2,2022-04-17,ATL,BOS,248.6
3,2022-04-17,ATL,BOS,248.6
4,2022-04-17,ATL,BOS,248.6


In [14]:


new_df.to_sql(table_name, engine, if_exists='append', index=True)



,flightDate,startingAirport,destinationAirport,isBasicEconomy,isRefundable,isNonStop,segmentsAirlineName,totalFare
0,2022-04-17,ATL,BOS,False,False,False,American Airlines||American Airlines,273
1,2022-04-17,ATL,BOS,False,False,False,Spirit Airlines||Spirit Airlines,301
2,2022-04-17,ATL,BOS,False,False,False,United||United,352
3,2022-04-17,ATL,BOS,False,False,True,American Airlines,355
4,2022-04-17,ATL,BOS,False,False,True,Delta,307


In [15]:


df = pd.read_sql(table_name, engine, index_col="index")
df.head()



flightDate             datetime64[ns]
startingAirport                object
destinationAirport             object
isBasicEconomy                   bool
isRefundable                     bool
isNonStop                        bool
segmentsAirlineName            object
totalFare                       int64
dtype: object

In [23]:
engine.dispose()